# Burgers Equation Identification Using ANN-LBFGS Optimizer
Equation:   $u_{t} + \lambda_1 uu_{x}-\lambda_2 u_{xx} = 0$  

In [45]:
#author : $um@nth
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
import scipy.io
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [46]:
N_u = 2500
layers = [2, 25, 25, 25, 25, 25, 25, 25, 25, 1]
data = scipy.io.loadmat('burgers_shock.mat') 
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))                  
u_star = Exact.flatten()[:,None]                                                 
lb = X_star.min(0)                                                              
ub = X_star.max(0)
np.random.seed(107)
idx = np.random.choice(X_star.shape[0], N_u, replace=False)

X_train = X_star[idx,:]
u_train = torch.from_numpy(u_star[idx,:]).float()
X = torch.from_numpy(X_train[:,0:1]).requires_grad_(True).float()     
T = torch.from_numpy(X_train[:,1:2]).requires_grad_(True).float()

In [47]:
class ANN(nn.Module):

  def __init__(self, layers):
    super(ANN, self).__init__()
    self.layers = nn.ModuleList()
    for i, j in zip(layers, layers[1:]):
      linear = nn.Linear(i, j)
      nn.init.xavier_normal_(linear.weight.data, gain = 1.0)
      nn.init.zeros_(linear.bias.data)
      self.layers.append(linear)
  
  def forward(self, x):
    L = len(self.layers)
    for l, transform in enumerate(self.layers):
      if l < L-1:
        x = torch.tanh(transform(x))
      else:
        x = transform(x)
    return x   

In [48]:
class ANN_Run():

  def __init__(self, X, T, model, u_train):
    self.X = X
    self.T = T
    self.u_train = u_train
    self.model = model
    self.i = 0
    self.optimizer = torch.optim.LBFGS(
            self.model.parameters(), 
            lr = 1, 
            max_iter = 20000, 
            max_eval = 20000, 
            history_size = 50,
            tolerance_grad = 1e-4, 
            tolerance_change = 1.0 * np.finfo(float).eps,
            line_search_fn = "strong_wolfe"
        )
  
  def closure(self):
    self.model.train()
    mse = nn.MSELoss()
    self.optimizer.zero_grad()
    yhat = self.model(torch.cat([self.X, self.T], axis=1).float())
    loss1 = mse(yhat, self.u_train)
    loss = loss1
    loss.backward()
    nu = 0.01/np.pi
    if self.i % 100 == 0:
      print('Epoch:', self.i, 'Loss: %.5e' % (loss.item()))
    self.i += 1
    return loss
  
  def train_(self):
    self.optimizer.step(self.closure)

In [49]:
ann_model = ANN(layers)
ann_ = ANN_Run(X, T, ann_model, u_train)
ann_.train_()

Epoch: 0 Loss: 7.10744e-01
Epoch: 100 Loss: 5.31385e-03
Epoch: 200 Loss: 4.75088e-04
Epoch: 300 Loss: 1.26425e-04


In [50]:
ann_model.eval()

ANN(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=25, bias=True)
    (1-7): 7 x Linear(in_features=25, out_features=25, bias=True)
    (8): Linear(in_features=25, out_features=1, bias=True)
  )
)

In [51]:
X_ = torch.from_numpy(X_star[:,0:1]).requires_grad_(True).float()     
T_ = torch.from_numpy(X_star[:,1:2]).requires_grad_(True).float()
xf = torch.cat([X_, T_], axis=1)
uf = ann_model(xf)
u_x = grad(uf.sum(), X_, retain_graph = True, create_graph = True)[0]
u_xx = grad(u_x.sum(), X_, retain_graph = True, create_graph = True)[0]
u_t = grad(uf.sum(), T_, retain_graph = True, create_graph = True)[0]

In [52]:
ind = abs(uf.detach().numpy()-u_star).reshape(-1).argsort()

In [53]:
A = np.hstack([(-uf*u_x).detach().numpy(), u_xx.detach().numpy()])
B = u_t.detach().numpy()

l_1, l_2 = np.linalg.lstsq(A, B, rcond=None)[0]

In [54]:
print('Lambda1 Pred:', round(l_1[0],8), '  ; Lambda1 Actual:', 1.0)
print('Lambda2 Pred:', round(l_2[0],8), '  ; Lambda2 Actual:', round(0.01/np.pi,8))

Lambda1 Pred: 0.82174206   ; Lambda1 Actual: 1.0
Lambda2 Pred: 0.00260338   ; Lambda2 Actual: 0.0031831
